In [1]:
%load_ext autoreload
%autoreload 2
import os
import os.path
import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
from sklearn.feature_extraction.text import CountVectorizer
from anchor import anchor_text
import time


In [2]:
# dataset from http://www.cs.cornell.edu/people/pabo/movie-review-data/
# Link: http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz
def load_polarity(path='/home/zjmiao/Documents/labelling_explanation/rt-polaritydata'):
    data = []
    labels = []
    f_names = ['rt-polarity.neg', 'rt-polarity.pos']
    for (l, f) in enumerate(f_names):
        for line in open(os.path.join(path, f), 'rb'):
            try:
                line.decode('utf8')
            except:
                continue
            data.append(line.strip())
            labels.append(l)
    return data, labels

Note: you must have spacy installed. Run:

        pip install spacy && python -m spacy download en_core_web_lg

If you want to run BERT, you can use the smaller spacy model, and you have to install transformers and torch or tf: 

        pip install torch transformers spacy && python -m spacy download en_core_web_sm
        

In [3]:
nlp = spacy.load('en_core_web_lg')
# Uncomment this if you're using BERT
# nlp = spacy.load('en_core_web_sm')

In [4]:
data, labels = load_polarity()
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(data, labels, test_size=.2, random_state=42)
train, val, train_labels, val_labels = sklearn.model_selection.train_test_split(train, train_labels, test_size=.1, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)

In [5]:
vectorizer = CountVectorizer(min_df=1)
vectorizer.fit(train)
train_vectors = vectorizer.transform(train)
test_vectors = vectorizer.transform(test)
val_vectors = vectorizer.transform(val)

In [15]:
c = sklearn.linear_model.LogisticRegression()
# c = sklearn.ensemble.RandomForestClassifier(n_estimators=500, n_jobs=10)
c.fit(train_vectors, train_labels)
preds = c.predict(val_vectors)
print('Val accuracy', sklearn.metrics.accuracy_score(val_labels, preds))
def predict_lr(texts):
    res = c.predict(vectorizer.transform(texts))
    print(texts, res)
    return res

Val accuracy 0.7544910179640718


/home/zjmiao/Documents/snorkel-tutorials/.envsnorkel/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Explaining a prediction
use_unk_distribution=True means we will perturb examples by replacing words with UNKS

In [10]:
explainer = anchor_text.AnchorText(nlp, [0, 1], use_unk_distribution=True)

In [16]:
np.random.seed(1)
text = 'This is a good book .'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
print('Prediction: %s' % pred)
exp = explainer.explain_instance(text, predict_lr, threshold=0.95)


['This is a good book .'] [1]
['This is a good book .'] [1]
Prediction: 1
['This is a good book .'] [1]
['This is a UNK book UNK'] [0]
['This UNK a good UNK .'] [1]
['UNK is a UNK UNK .'] [0]
['This is UNK good book UNK', 'This is a good book .', 'This UNK UNK good book UNK', 'This UNK a good UNK .', 'This is a good UNK .', 'This UNK a good UNK .', 'This is a UNK UNK UNK', 'This is UNK UNK book .', 'This is a UNK book UNK', 'This UNK a UNK book .'] [1 1 1 1 1 1 0 0 0 0]
['UNK is UNK good UNK UNK', 'This is a good UNK .', 'This UNK a good UNK UNK', 'This is a good UNK UNK', 'This is a good book UNK', 'UNK is a good book UNK', 'UNK UNK a good UNK UNK', 'This is UNK good book .', 'UNK is a good book .', 'This UNK a good book .'] [1 1 1 1 1 1 1 1 1 1]
['UNK is a good book UNK', 'UNK is a good UNK UNK', 'This is a good UNK UNK', 'UNK is UNK UNK UNK .', 'UNK is UNK UNK UNK .', 'UNK is a good book UNK', 'UNK is a good UNK UNK', 'UNK is UNK good book UNK', 'This is a UNK UNK UNK', 'This is a g

Let's take a look at the anchor. Note that using this perturbation distribution, having the word 'good' in the text virtually guarantees a positive prediction

In [12]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))


Anchor: good
Precision: 1.00

Examples where anchor applies and model predicts 1:

This UNK UNK good UNK .
UNK is a good UNK .
This UNK UNK good book .
UNK UNK a good book .
UNK UNK UNK good book .
This is UNK good book .
UNK is UNK good book UNK
UNK is UNK good UNK UNK
UNK is a good book .
UNK UNK a good UNK UNK

Examples where anchor applies and model predicts 0:




### Changing the distribution
Let's try this with another perturbation distribution, namely one that replaces words by similar words instead of UNKS, using word embeddings.

In [10]:
explainer = anchor_text.AnchorText(nlp, ['negative', 'positive'], use_unk_distribution=False, use_bert=False)

In [11]:
np.random.seed(1)
text = 'This is a good book .'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
print('Prediction: %s' % pred)
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False, use_proba=True)

Prediction: positive


Let's take a look at the anchor now. Note that with this distribution, we need more to guarantee a prediction of positive.

In [12]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Anchor: good AND book AND is
Precision: 1.00

Examples where anchor applies and model predicts positive:

THIS is the good book .
THis is each good book .
Another is an good book .
Both is both good book .
Every is the good book .
A is an good book .
THESE is an good book .
THIS is a good book .
THis is a good book .
ANOTHER is some good book .

Examples where anchor applies and model predicts negative:




Let's take a look at the partial anchor 'good' to see why it's not sufficient in this case


In [13]:
print('Partial anchor: %s' % (' AND '.join(exp.names(0))))
print('Precision: %.2f' % exp.precision(0))
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Partial anchor: good
Precision: 0.85

Examples where anchor applies and model predicts positive:

This refers any good book .
SOME sits a good lesson .
A believes a good synopsis .
The brings a good preface .
Any suggests an good cookbook .
THe is the good poetry .
Every appears a good synopsis .
Both makes this good publication .
THis looks this good hardcover .
Those becomes an good autobiography .

Examples where anchor applies and model predicts negative:

A falls another good story .
This falls any good paperback .
Another contains some good publisher .
This reminds a good issue .
THIS wants another good fantasy .
Some creates the good everything .
The goes a good textbook .
Some remains every good lecture .
SOME occurs any good biography .
SOME has every good textbook .


## Using BERT

The distribution above is a bit naive, and you get a lot of sentences that don't look realistic.  
Let's use BERT to perturb the data:

In [14]:
explainer = anchor_text.AnchorText(nlp, ['negative', 'positive'], use_unk_distribution=False, use_bert=True)

/home/zjmiao/Documents/snorkel-tutorials/.envsnorkel/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [15]:
np.random.seed(1)
text = 'This is a good book .'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
print('Prediction: %s' % pred)
b = time.time()
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False)
print('Time: %s' % (time.time() - b))

Prediction: positive
Time: 41.04053235054016


In [21]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Anchor: good AND book AND is
Precision: 0.99

Examples where anchor applies and model predicts positive:

This is a good book .
What is a good book .
this is a good book award
it is another good book .
this is a good book ?
Your is damn good book .
Allah is your good book .
here is always good book editor
This is a good book .
Help is a good book review

Examples where anchor applies and model predicts negative:

everything is a good book !


Let's take a look at the partial anchor 'good' to see why it's not sufficient in this case


In [22]:
print('Partial anchor: %s' % (' AND '.join(exp.names(0))))
print('Precision: %.2f' % exp.precision(0))
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Partial anchor: good
Precision: 0.88

Examples where anchor applies and model predicts positive:

hers is a good fare .
• are a good at !
Where ##s a good photo student
here is said good word ;
it is a good growth .
music is a good job .
" = become good rid •
they ##um your good luck .
for do · good di :
it is a good luck movie

Examples where anchor applies and model predicts negative:

weather is not good behavior .
we needs a good bath ;
there is a good neighbourhood affair
Neither indicates despite good news .
thirst is a good idea !
this is a good animal .
" is very good horse !
God is also good weather .
) , a good strategy edition
only . ‘ good ##will >


Note how the examples are much more varied than just using word embeddings.

We are sampling from BERT sequentially above (one mask at a time), to get sentences that are more coherent.  
If you want to do a single BERT pass per sample, you can set `onepass=True`. You may lose some coherence, but it will be much faster.

In [23]:
np.random.seed(1)
text = 'This is a good book .'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
print('Prediction: %s' % pred)
b = time.time()
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False, onepass=True)
print('Time: %s' % (time.time() - b))

Prediction: positive
Time: 1.6398329734802246


In [24]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Anchor: good AND book AND is
Precision: 0.99

Examples where anchor applies and model predicts positive:

This is a good book .
What is a good book .
this is a good book award
it is another good book .
this is a good book ?
Your is damn good book .
Allah is your good book .
here is always good book editor
This is a good book .
Help is a good book review

Examples where anchor applies and model predicts negative:

everything is a good book !


Let's take a look at the partial anchor 'good' to see why it's not sufficient in this case


In [25]:
print('Partial anchor: %s' % (' AND '.join(exp.names(0))))
print('Precision: %.2f' % exp.precision(0))
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=2, only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=2, only_different_prediction=True)]))

Partial anchor: good
Precision: 0.88

Examples where anchor applies and model predicts positive:

This is a good book .
What is a good book .
this is a good book award
it is another good book .
this is a good book ?
Your is damn good book .
Allah is your good book .
here is always good book editor
This is a good book .
Help is a good book review

Examples where anchor applies and model predicts negative:

everything is a good book !


## See a visualization of the anchor with examples and etc (won't work if you're seeing this on github)

In [ ]:
exp.show_in_notebook()